<a href="https://colab.research.google.com/github/isc-patrick/Edifai-AI-Class/blob/main/Data%2C_best_practices%2C_python_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps to prepare for InstallFest - Rough Draft

I think this structure is what created a lot of work for the last class as a workspace/account has to be create for every team. This probably can only be avoided(or made easier) by having an Enterprise level account with HuggingFace

1. Create 20 Accounts, one for each team: Team11, Team12,... Team15, Team21, ...Team54
  2. This requires an edifai email for each team and a password.



Collab notebooks are ephemeral, both their compute and storage. The folder icon in the left menu opens your local ephemeral storage directory. For non-ephemeral storage we are using hugging face datasets and provide some functions to load and save files that you want persisted. For any intermediate data files that you do not want to persist for longer than the current session, you should use the local file system.  Note that Collab sessions can end at anytime forcing you to reconnect to a new process.


A Dataset in huggingface is both a git repo and a class. The Dataset class can load a repo, there is typically a 1-1 reltaionship. However, we are also using Datasets as a persistent store. So be aware that the term dataset can refer to the dataset repo, https://huggingface.co/docs/datasets/en/upload_dataset#create-a-repository, or a dataset object, https://huggingface.co/docs/datasets/en/index.


# Step 1
Create an access token for your HuggingFace account. Log into Hugging Face and go to this page and create a write token, https://huggingface.co/settings/tokens. Add this as a Secret in the notebook with the name HF_TOKEN.

In [7]:
# Step 2 - CHECK ACCESS TOKEN

import os
from huggingface_hub import HfApi
from huggingface_hub import login
from google.colab import userdata

# Check access token
login(userdata.get('HF_TOKEN'))
api = HfApi()

# Do this on init, will use later
REPO_PREFIX = "patjs/"
LOCAL_ROOT_DIR = "patient1"
repo_id = f"{REPO_PREFIX}{LOCAL_ROOT_DIR}"

# Throws an exception if it does not exist or have no access
api.auth_check(repo_id=BASE_REPO, repo_type="dataset")

print("looks good")

looks good


In [26]:
# STEP 3 - create a dataset repo, check it and then delete


from huggingface_hub import create_repo, delete_repo
new_repo = f"{REPO_PREFIX}{LOCAL_ROOT_DIR}"

repo_url = create_repo(new_repo, repo_type="dataset")
print(f" Created {repo_url}")


 Created https://huggingface.co/datasets/patjs/mydata


In [27]:
delete_repo(new_repo, repo_type="dataset")
print(f" Deleted {repo_url}")


 Deleted https://huggingface.co/datasets/patjs/mydata


In [10]:
# Load and Save File functions

from huggingface_hub import login
from google.colab import userdata
import os

login(userdata.get('HF_TOKEN'))

from huggingface_hub import HfApi
api = HfApi()

def load_file(filename, foldername) -> str:
  """
  load_file takes a filename and foldername for a hugging face dataset, loads the file into the collab file system
  and returns the filepath. The filepath is returned instead of the file content so it can be used in other functions that
  load data via a local file path, like pandas.read_csv(file_path).

  """
  filepath =  api.hf_hub_download(repo_id=repo_id, filename=filename, subfolder=foldername, repo_type="dataset")

  with open(filepath, 'r') as fi:
    data = fi.read()

  local_dir = os.path.join(repo_id,foldername)

  # create the dir if it doesn't exist
  if not os.path.exists(local_dir):
    os.makedirs(local_dir, exist_ok=True)

  local_path = os.path.join(repo_id,foldername,filename)

  with open(local_path, 'w') as fo:
    fo.write(data)

  return local_path


def save_file(file_obj, file_name):
  api.upload_file(
      path_or_fileobj=file_obj,
      path_in_repo=file_name,
      repo_id=repo_id,
      repo_type="dataset",
  )

In [ ]:
""" Some Python topics
    - Pandas vs HF Datasets
    - Git vs HTTP for repos
      - https://huggingface.co/docs/huggingface_hub/en/concepts/git_vs_http


"""

In [ ]:
"""
Jupyter best practices and tips
  - Use cells to quickly get working code and refactor into functions and classes as notebook grows
  - If you experience "weird" behavior, restarting the kernel is a good place to start
    - old variables, variable name changes can make the working memory hide errors no longer visible in code and have other side effects
  - Use hot keys. Jupyter is primarily about speed of iteration and presentation
  - If you want to reuse code, but don't want to copy and paste code into different notebooks you can save to a file and import it.

"""

In [1]:
%%writefile my_script.py
# This is a Python script
def my_function():
    print("Hello from my_function!")

my_function()


Overwriting my_script.py


In [8]:
# Note that this will only be imported once so you will have to restart kernal to see the print from my_function
import my_script

# Save the file to HF dataset persistence
save_file("./my_script.py", "my_script.py")

In [11]:

# Restart the kernal then run the # Step 2 - CHECK ACCESS TOKEN And the # Load and Save File functions and the cell and then this cell
load_file("my_script.py", "")
import my_script

Hello from my_function!


In [ ]:
# Remembering what cells to run is not the right way
# Open this refactored version with all code put into functions and then create cells to show workflows